# QIIME 2 enables comprehensive end-to-end analysis of diverse microbiome data and comparative studies with publicly available data

this is a QIIME 2 Artifact CLI notebook which replicated analyses in the QIIME 2 protocol

**environment:** qiime2-2020.2

Use this version of [q2-classo](https://github.com/Leo-Simpson/q2-classo/tree/edbf9d20fed931cbc1cff77f5634bb0dcd783c8a) 

In [1]:
pip install --upgrade c-lasso

     |████████████████████████████████| 56 kB 934 kB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install c-lasso

Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install zarr

Note: you may need to restart the kernel to use updated packages.


In [5]:
pip install plotly

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install c-lasso

pip install zarr

pip install plotly

SyntaxError: invalid syntax (<ipython-input-2-4d321bd1b920>, line 1)

In [7]:
cd ..

/data


In [8]:
!python setup.py install

running install
running bdist_egg
running egg_info
writing q2_classo.egg-info/PKG-INFO
writing dependency_links to q2_classo.egg-info/dependency_links.txt
writing entry points to q2_classo.egg-info/entry_points.txt
writing top-level names to q2_classo.egg-info/top_level.txt
reading manifest file 'q2_classo.egg-info/SOURCES.txt'
writing manifest file 'q2_classo.egg-info/SOURCES.txt'
installing library code to build/bdist.linux-x86_64/egg
running install_lib
running build_py
creating build/bdist.linux-x86_64
creating build/bdist.linux-x86_64/egg
creating build/bdist.linux-x86_64/egg/q2_classo
copying build/lib/q2_classo/_dict.py -> build/bdist.linux-x86_64/egg/q2_classo
copying build/lib/q2_classo/_tree.py -> build/bdist.linux-x86_64/egg/q2_classo
creating build/bdist.linux-x86_64/egg/q2_classo/.ipynb_checkpoints
copying build/lib/q2_classo/.ipynb_checkpoints/_func-checkpoint.py -> build/bdist.linux-x86_64/egg/q2_classo/.ipynb_checkpoints
creating build/bdist.linux-x86_64/egg/q2_classo/_

In [9]:
!pip install -e .

Obtaining file:///data
  Attempting uninstall: q2-classo
    Found existing installation: q2-classo 0.0.0.dev0
    Uninstalling q2-classo-0.0.0.dev0:
      Successfully uninstalled q2-classo-0.0.0.dev0
  Running setup.py develop for q2-classo


In [10]:
!qiime dev refresh-cache

QIIME is caching your current deployment for improved performance. This may take a few moments and should only happen once per deployment.


In [11]:
cd example/data

/data/example/data


## Filter features

In [13]:
!qiime feature-table filter-features \
  --i-table table.qza \
  --p-min-samples 20 \
  --o-filtered-table filtered-table.qza

Saved FeatureTable[Frequency] to: filtered-table.qza


## log-contrast and taxa processing

Either collapse at genus level, which is the 'easy way', but not really what we want

In [14]:
!qiime taxa collapse --i-table table.qza \
  --i-taxonomy taxonomy.qza \
  --p-level 6 \
  --o-collapsed-table genus_table.qza

Saved FeatureTable[Frequency] to: genus_table.qza


In [15]:
!qiime classo transform-features \
     --p-transformation clr \
     --p-coef 0.5 \
     --i-features genus_table.qza \
     --o-x genus_table_clr

Saved FeatureTable[Design] to: genus_table_clr.qza


In [16]:
 !qiime classo transform-features \
     --p-transformation clr \
     --p-coef 0.5 \
     --i-features filtered-table.qza \
     --o-x xclr

Saved FeatureTable[Design] to: xclr.qza


In [17]:
!qiime classo add-taxa \
	--i-features xclr.qza  \
	--i-taxa taxonomy.qza \
	--o-x xtaxa --o-aweights wtaxa

Saved FeatureTable[Design] to: xtaxa.qza
Saved Weights to: wtaxa.qza


In [18]:
!qiime classo regress --help

Usage: qiime classo regress [OPTIONS]

  The function computes the constrainted_sparse_regression vector with
  respect to the formulation of regression that is asked and with respect to
  the model selection parameters given

Inputs:
  --i-features ARTIFACT   Matrix representing the data of the problem
    FeatureTable[Design]                                            [required]
  --i-c ARTIFACT          Constraint matrix, default is the zero-sum
    ConstraintMatrix                                                [optional]
  --i-weights ARTIFACT    Vector of weights for penalization
    Weights                                                         [optional]
Parameters:
  --m-y-file METADATA
  --m-y-column COLUMN  MetadataColumn[Numeric]
                          Vector representing the output of the problem
                                                                    [required]
  --p-do-yshift / --p-no-do-yshift
                          if set to true, then y will be cent

## Add covariates

In [19]:
!qiime classo add-covariates \
    --i-features xtaxa.qza \
    --i-weights wtaxa.qza \
    --m-covariates-file sample-metadata-complete.tsv \
    --p-to-add host_sexual_orientation host_age host_body_mass_index ethnicity \
    --p-w-to-add 0.1 1. 1. 0.1 \
    --o-new-features xcovariates \
    --o-new-c ccovariates \
    --o-new-w wcovariates

Saved FeatureTable[Design] to: xcovariates.qza
Saved ConstraintMatrix to: ccovariates.qza
Saved Weights to: wcovariates.qza


## Split table

Split data into training and testing sets : 

In [20]:
!qiime sample-classifier split-table \
	--i-table xcovariates.qza \
	--m-metadata-file sample-metadata-complete.tsv \
	--m-metadata-column sCD14  \
	--p-test-size 0.2 \
	--p-random-state 42 \
	--p-stratify False \
	--o-training-table regress-xtraining \
	--o-test-table regress-xtest

Saved FeatureTable[Design] to: regress-xtraining.qza
Saved FeatureTable[Design] to: regress-xtest.qza


In [21]:
!qiime sample-classifier split-table \
	--i-table xcovariates.qza \
	--m-metadata-file sample-metadata-complete.tsv \
	--m-metadata-column HIV_serostatus  \
	--p-test-size 0.2 \
	--p-random-state 42 \
	--p-stratify False \
	--o-training-table classify-xtraining \
	--o-test-table classify-xtest

Saved FeatureTable[Design] to: classify-xtraining.qza
Saved FeatureTable[Design] to: classify-xtest.qza


## Regression task 

Apply classo to the training set to solve the linear regression problem : 

In [22]:
!qiime classo regress  \
    --i-features regress-xtraining.qza \
    --i-c ccovariates.qza \
    --i-weights wcovariates.qza \
    --m-y-file sample-metadata-complete.tsv \
    --m-y-column sCD14  \
    --p-concomitant \
    --p-stabsel \
    --p-cv \
    --p-path \
    --p-lamfixed \
    --p-stabsel-threshold 0.5 \
    --p-cv-seed 1 \
    --p-no-cv-one-se \
    --o-result regresstaxa

Saved CLASSOProblem to: regresstaxa.qza


## Classification task

In [23]:
!qiime classo classify  \
    --i-features classify-xtraining.qza \
    --i-c ccovariates.qza \
    --i-weights wcovariates.qza \
    --m-y-file sample-metadata-complete.tsv \
    --m-y-column HIV_serostatus  \
    --p-huber \
    --p-stabsel \
    --p-cv \
    --p-path \
    --p-lamfixed \
    --p-stabsel-threshold 0.5 \
    --p-cv-seed 42 \
    --p-no-cv-one-se \
    --o-result classifytaxa

Saved CLASSOProblem to: classifytaxa.qza


## Prediction 

In [24]:
!qiime classo predict \
    --i-features regress-xtest.qza \
    --i-problem regresstaxa.qza \
    --o-predictions regress-predictions.qza

Saved CLASSOProblem to: regress-predictions.qza


In [25]:
!qiime classo predict \
    --i-features classify-xtest.qza \
    --i-problem classifytaxa.qza \
    --o-predictions classify-predictions.qza

Saved CLASSOProblem to: classify-predictions.qza


## Visualization

In [26]:
!qiime classo summarize \
  --i-problem regresstaxa.qza \
  --i-taxa taxonomy.qza \
  --i-predictions regress-predictions.qza \
  --o-visualization regresstaxa.qzv

Saved Visualization to: regresstaxa.qzv


In [27]:
!qiime classo summarize \
  --i-problem classifytaxa.qza \
  --i-taxa taxonomy.qza \
  --i-predictions classify-predictions.qza \
  --o-visualization classifytaxa.qzv \
  --verbose

/data/q2_classo/_summarize/_visualizer.py:772: FutureWarning:

Index.__and__ operating as a set operation is deprecated, in the future this will be a logical operation matching Series.__and__.  Use index.intersection(other) instead

/data/q2_classo/_summarize/_visualizer.py:773: FutureWarning:

Index.__and__ operating as a set operation is deprecated, in the future this will be a logical operation matching Series.__and__.  Use index.intersection(other) instead

/data/q2_classo/_summarize/_visualizer.py:772: FutureWarning:

Index.__and__ operating as a set operation is deprecated, in the future this will be a logical operation matching Series.__and__.  Use index.intersection(other) instead

/data/q2_classo/_summarize/_visualizer.py:773: FutureWarning:

Index.__and__ operating as a set operation is deprecated, in the future this will be a logical operation matching Series.__and__.  Use index.intersection(other) instead

/data/q2_classo/_summarize/_visualizer.py:772: FutureWarning:

Index

In [28]:
!qiime tools view regresstaxa.qzv

Usage: qiime tools view [OPTIONS] VISUALIZATION

Error: Visualization viewing is currently not supported in headless environments. You can view Visualizations (and Artifacts) at https://view.qiime2.org, or move the Visualization to an environment with a display and view it with `qiime tools view`.


In [29]:
!qiime tools view classifytaxa.qzv

Usage: qiime tools view [OPTIONS] VISUALIZATION

Error: Visualization viewing is currently not supported in headless environments. You can view Visualizations (and Artifacts) at https://view.qiime2.org, or move the Visualization to an environment with a display and view it with `qiime tools view`.


Alternatively, one can drag&drop the file problemtaxa.qzv on : https://view.qiime2.org
Thanks to this alternative, one can also track the workflow that the qiime2 artifact did. 

In [32]:
print("hello classo!")

hello classo!
